<a href="https://colab.research.google.com/github/an-karpov/llms/blob/main/notebooks/saiga_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import transformers